In [26]:
import PIL
import os
from PIL import Image
import pandas as pd
from numpy import asarray
import numpy as np

In [27]:
def loadImages(path):
    imagesList = os.listdir(path)
    loadedImages = []
    loadfilenames = []
    loaddata = []
    
    for image in imagesList:
        loadfilenames.append(image)
        
        with open(os.path.join(path, image), 'rb') as i:
            img = Image.open(i)            
            loadedImages.append(img)
            data = asarray(img)
            loaddata.append(data)
            
    return loadedImages, loadfilenames, loaddata

In [28]:
loadedImages, loadfilenames, loaddata = loadImages('images')

In [29]:
consolidated_X = np.array(loaddata)

In [30]:
consolidated_X.shape

(2352, 224, 224, 3)

In [31]:
# now get the label values for the images
train_df = pd.read_csv('train.csv')

In [32]:
train_df.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [33]:
loadfilenames = pd.DataFrame(loadfilenames, columns = ['image_names'])

In [34]:
loadfilenames.head()

,image_names
0,0.jpg
1,1.jpg
2,10.jpg
3,100.jpg
4,1000.jpg


In [35]:
loadfilenames.to_csv('final_load.csv')

In [36]:
train_y = pd.merge(loadfilenames, train_df, how='left')

In [37]:
train_y.head()

,image_names,emergency_or_not
0,0.jpg,1.0
1,1.jpg,1.0
2,10.jpg,1.0
3,100.jpg,NaN
4,1000.jpg,NaN


In [38]:
test_index = train_y[train_y['emergency_or_not'].isna()].index
train_index = train_y[~train_y['emergency_or_not'].isna()].index

In [39]:
train_x = consolidated_X[train_index]
train_y = np.array(train_y.loc[train_index]['emergency_or_not'])

In [40]:
from sklearn.model_selection import train_test_split

x1,x2,y1,y2 = train_test_split(train_x, train_y, test_size=0.3)

In [41]:
x1.shape

(1152, 224, 224, 3)

In [42]:
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model

In [43]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)

restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
    
restnet.summary()

C:\Users\91965\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

C:\Users\91965\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  


In [44]:
restnet.trainable = True
set_trainable = False
for layer in restnet.layers:
    if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in restnet.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at...,input_2,False
1,<keras.layers.convolutional.ZeroPadding2D obje...,conv1_pad,False
2,<keras.layers.convolutional.Conv2D object at 0...,conv1,False
3,<keras.layers.normalization.BatchNormalization...,bn_conv1,False
4,<keras.layers.core.Activation object at 0x0000...,activation_50,False
...,...,...,...
171,<keras.layers.convolutional.Conv2D object at 0...,res5c_branch2c,True
172,<keras.layers.normalization.BatchNormalization...,bn5c_branch2c,True
173,<keras.layers.merge.Add object at 0x0000024AAB...,add_32,True
174,<keras.layers.core.Activation object at 0x0000...,activation_98,True


In [45]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model_finetuned = Sequential()
model_finetuned.add(restnet)
model_finetuned.add(Dense(512, activation='relu', input_dim=100352))
model_finetuned.add(Dropout(0.3))
model_finetuned.add(Dense(512, activation='relu'))
model_finetuned.add(Dropout(0.3))
model_finetuned.add(Dense(1, activation='sigmoid'))
model_finetuned.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-7),
              metrics=['accuracy'])
model_finetuned.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 100352)            23587712  
_________________________________________________________________
dense_4 (Dense)              (None, 512)               51380736  
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 513       
Total params: 75,231,617
Trainable params: 55,059,457
Non-trainable params: 20,172,160
_________________________________

In [46]:
from keras.preprocessing.image import ImageDataGenerator

In [47]:
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255, featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = datagen.flow(train_x, train_y, subset='training')
val_generator = datagen.flow(train_x, train_y, subset='validation')

In [48]:
model_finetuned.fit(train_x, train_y, batch_size=50, epochs=1, validation_split=0.2, verbose=1)

Train on 1316 samples, validate on 330 samples
Epoch 1/1
1316/1316 [==============================] - 422s 321ms/step - loss: 0.8238 - accuracy: 0.5866 - val_loss: 1.4483 - val_accuracy: 0.1485


## Storing the Augumented images just for only one image.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('images/0.jpg')  
x = img_to_array(img)  
x = x.reshape((1,) + x.shape)  

print(x.shape)
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='vehichle', save_format='jpeg'):
    i += 1
    if i > 19:
        break  # otherwise the generator would loop indefinitely